@book{advanced_physics, author = {Stephen Hawking and Brian Greene and Maria Trump and Susan Soy}, title = {Advanced Concepts in Theoretical Physics}, publisher = {Cambridge University Press}, year = {2025}, edition = {3}, volume = {1}, series = {Advanced Studies in Physics}, address = {Cambridge, UK}, month = {May}, note = {A seminal work in the field of theoretical physics}, isbn = {978-0-521-76948-0}, doi = {10.1017/example.book.2025}, url = {https://www.cambridge.org/advanced_physics}, annote = {Widely cited in the physics community}, abstract = {This book explores cutting-edge theories and concepts in theoretical physics...}, keywords = {Theoretical Physics, Quantum Mechanics, String Theory}, language = {English}, price = {75.00}, size = {600 pages}, lccn = {2020934576}, mrnumber = {MR3070071} }

@article{quantum_entanglement,
  author        = {Albert Einstein and Boris Podolsky and Nathan Rosen},
  title         = {Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?},
  journal       = {Physical Review},
  year          = {1935},
  volume        = {47},
  number        = {10},
  pages         = {777-780},
  month         = {May},
  note          = {EPR Paradox paper, fundamental for quantum mechanics},
  doi           = {10.1103/PhysRev.47.777},
  url           = {https://journals.aps.org/pr/abstract/10.1103/PhysRev.47.777},
  abstract      = {In this paper, the authors discuss the EPR paradox and challenge the completeness of quantum mechanics...},
  keywords      = {Quantum Mechanics, EPR Paradox, Physical Reality},
  language      = {English},
  publisher     = {American Physical Society}
}

@inproceedings{deep_learning,
  author        = {Geoffrey Hinton and Yoshua Bengio and Yann LeCun},
  title         = {Deep Learning for Artificial Intelligence},
  booktitle     = {Proceedings of the IEEE International Conference on Neural Networks},
  year          = {2021},
  editor        = {Jane Smith and John Doe},
  volume        = {1},
  number = {5},
  series        = {Advances in Neural Information Processing},
  pages         = {100-120},
  address       = {Montreal, Canada},
  month         = {December},
  organization  = {IEEE},
  publisher     = {IEEE Press},
  note          = {Keynote paper on recent advancements in deep learning},
  isbn          = {978-1-5386-4637-1},
  doi           = {10.1109/ICNN.2021.9483948},
  url           = {https://ieeexplore.ieee.org/document/9483948},
  annote        = {A seminal work on how deep learning transforms AI},
  abstract      = {This paper explores cutting-edge deep learning techniques and their impact on the development of artificial intelligence...},
  keywords      = {Deep Learning, Artificial Intelligence, Neural Networks},
  language      = {English}
}


@incollection{quantum_computation,
  author        = {Michael A. Nielsen and Isaac L. Chuang},
  title         = {Quantum Computation and Quantum Information},
  booktitle     = {Handbook of Quantum Information Science},
  publisher     = {Springer},
  year          = {2026},
  editor        = {Charles H. Bennett and David P. DiVincenzo},
  volume        = {4},
  series        = {Quantum Science and Technology},
  chapter       = {10},
  pages         = {250-300},
  address       = {Berlin, Germany},
  month         = {October},
  note          = {A comprehensive overview of the fundamentals of quantum computation},
  isbn          = {978-3-540-88702-7},
  doi           = {10.1007/springerreference-303198},
  url           = {https://www.springer.com/gp/book/9783540887027},
  annote        = {Essential reading for researchers entering the field of quantum information},
  abstract      = {This chapter delves into the principles of quantum computing, offering an accessible yet thorough introduction...},
  keywords      = {Quantum Computing, Quantum Information, Computational Models},
  language      = {English},
  price         = {45.00},
  size          = {50 pages}
}


In [24]:
import torch
import pandas as pd
torch.cuda.is_available()
from transformers import pipeline
import re
import string
import spacy
import requests

def custom_strip(text, replaceCharacter = []):
    
    '''
    Strip-Funktion, die standardmäßig neben Whitespace auch zeichen aus string.punctuation und “ sowie ” entfernt werden.
    
    Parameter:
    text: Text, der gestripped werden soll
    replaceCharacter = []: Liste von Zeichen, die aus der standardmäßigen Prüfung entfernt werden sollen.
    
    return: gestrippter Text
    '''
    
    allowed_chars = string.punctuation + string.whitespace + "“" + "”"
    for character in replaceCharacter:
        allowed_chars = allowed_chars.replace(character, '')
    return text.strip(allowed_chars)


def getIndexOfSubstring(text, regEx = [], reverse = False):
    
    '''
    Prüft für eine Liste von RegEx, ob sie in text vorkommen. Der RegEx, der den Substring mit der grötßen
    Länge ermittelt, kommt zum Zuge.
    
    Parameter:
    text: Text, wo das Auftreten des RegEx geprüft wird
    regEx = []: Liste mit RegEx. 
    reverse: Ob das erste oder letzte Auftreten eines Matches geprüft maßgeblich ist. Wenn reverse = False, dann wird das
    erste Auftreten geprüft.
    
    return: Substring und den zugehörigen Start- und Endindex.
    '''
    
    length = 0
    matches = []
    substring = ""
    #print(f'regEx: {regEx}')
    #print(f'text: {text}')
    for regExElement in regEx:
        matches = list(re.finditer(regExElement, text))
        #print(f'matches: {matches}')
        if matches:
            if reverse:
                match = matches[-1]
            else:
                match = matches[0]
            buffer = match.end() - match.start()
            if buffer > length:
                length = buffer
                startIndex = match.start()
                endIndex = match.end()
                substring = text[match.start():match.end()]
    if substring != "":
        return startIndex, endIndex, substring   
    return -1, -1, substring

def replaceSubstring (startIndex, endIndex, text, substituteString, ignorePunctuation = ["&", "(", ")"]):
    
    '''
    Ersetzt in einem String einen Substring durch einen anderen Substring (substituteString). Wenn der zu ersetztende 
    Substring mit bestimmten Zeichen beginnt oder endet und diese bestimmten Zeichen nicht durch ignorePunctuation 
    ignoriert werden sollen, dann wird der StartIndex runtergezählt und der EndIndex hochgezählt. 
    
    Parameter:
    startIndex: Index, wo der zu ersetztende Substring in Text eingefügt werden soll
    endIndex: Index, wo der zu ersetztende Substring in Text enden soll
    text: Text, wo das Auftreten des Substrings geprüft wird
    substituteString: Der einzufügende Substring
    ignorePunctuation: Zeichen, die für die Indexverschiebung nicht berücksichtigt werden sollen
    reverse: Ob das erste oder letzte Auftreten eines Matches geprüft werden soll. Wenn reverse = False, dann wird das
    erste Auftreten geprüft.
    
    return: Substring und den zugehörigen Start- und Endindex.
    '''
    
    if endIndex > 0:
        startIndexReplace = 0
        endIndexReplace = 0
        if startIndex > 0:
            for i in range(startIndex, -1, -1):
                if isSpeceficPunctuation(text[i], ignorePunctuation):
                    startIndexReplace = i + 1
                    break
        else:
            startIndexReplace = 0            
        #print(f' replaceSubstring, startIndexReplace={{{startIndexReplace}}}')
        if endIndex < len(text):
            for i in range(endIndex-1, len(text), 1):
                if isSpeceficPunctuation(text[i], ignorePunctuation):
                    endIndexReplace = i + 1
                    break
                elif i == len(text)-1:
                    endIndexReplace = len(text)
        else:
            endIndexReplace = len(text)
        #print(f' replaceSubstring, endIndexReplace={{{endIndexReplace}}}')
        if endIndexReplace > 0:
            changedText = text[0:startIndexReplace] + substituteString + text[endIndexReplace:len(text)]
            return changedText, text[startIndexReplace:endIndexReplace]
    return text, ""

def is_SurenameFirst(names):
    
    '''
    Prüft, ob die Autoren- oder Editornamen mit dem Vornamen beginnen. 
    
    Parameter:
    names: Substring vom Literaturstring, der die Autoren- oder Editornamen enthält
    '''
    
    splitedNames = names.split(" ")
    #regex wie w+ erkennt bspw. KEIN è 
    if splitedNames[0].endswith("."):
        return True
    splitedNames = names.split(",")
    if all(" " in item.strip() for item in splitedNames):
        return True
    return False
    
def is_NameShortened(df_PER):
    
    '''
    Prüft, ob die Namen mit einem Punkt abgekürzt sind
    
    Parameter:
    df_Per: Dataframe mit den Start- und Endindizes von Personennamen
    '''
    
    for index in df_PER.index.values.tolist():
        if "." == text[df_PER["end"].iloc[index]] and len(text[df_PER["start"].iloc[index]:df_PER["end"].iloc[index] + 1]) == 2:
            return True
    return False


def isSpeceficPunctuation(text, replaceCharacter = []):
    
    '''
    Prüft, ob ein Sring nur aus bestimmten Satzzeichen besteht. Standardmäßig wird string.punctuation + string.whitespace
    geprüft.
    
    Parameter:
    text: Text, der geprüft werden soll
    replaceCharacter = []: Liste von Zeichen, die aus der standardmäßigen Prüfung entfernt werden sollen.
    '''
        
    allowed_chars = string.punctuation + string.whitespace
    for character in replaceCharacter:
        allowed_chars = allowed_chars.replace(character, '')
    return all(char in allowed_chars for char in text)

def is_Editor(editorRegEx, textBetweenNames, startIndexTextBetweenNames, markerBehind = True):
    
    '''
    Prüft, ob es sich bei einem String um ein Signalwort für Editoren handelt.
    
    Parameter:
    editorRegEx: RegEx, die Signalwörter für das Auftreten von Editoren erkennen sollen
    textBetweenNames: Substring vom Literaturstring, der geprüft weden soll, ob das Signalwort enthalten ist. Dieser 
    Substring steht stehts zwischen potententiellen Namen.
    startIndexTextBetweenNames: Startindex, wo Substring im original Literaturstring steht
    markerBehind: Ob das Signalwort für Editoren vor oder hinter den Editorennamen im original Literaturstring steht
    
    return: Boolean, ob gefunden, und Start- und Endindizies, wo es im original Literaturstring vorkommt
    '''
    
    startSubstring, endSubstring, substring = getIndexOfSubstring(textBetweenNames, [editorRegEx])
    if startIndexTextBetweenNames > -1 and markerBehind:
        if isSpeceficPunctuation(textBetweenNames[startIndexTextBetweenNames:startSubstring], ["&"]):
            return True, startSubstring + startIndexTextBetweenNames, endSubstring + startIndexTextBetweenNames
    elif startIndexTextBetweenNames > -1:
        if isSpeceficPunctuation(textBetweenNames[endSubstring:startIndexTextBetweenNames], ["&"]):
            return True, startSubstring + startIndexTextBetweenNames, endSubstring + startIndexTextBetweenNames
    return False, -1, -1

def processNames(authors):
    
    '''
    Bereitet die Autorennamen in ein Standardformat auf
    
    Parameter:
    authors: Substring aus dem original Literaturstring, der die Autoren enthält
    
    return: Standardformat für Autoren
    '''
    
    finalAuthors = ""
    search_terms = [" and ", ", and ", " & ", ", & "]
    surenameFirst = is_SurenameFirst(authors.strip())
    authors = custom_strip(authors)
    if surenameFirst:
        startIndex, endIndex, andInAuthors = getIndexOfSubstring(authors, search_terms)
        if startIndex >= 0:
            authors = authors.replace(andInAuthors, " and ")
            finalAuthors = authors.replace(", ", " and ")
        else:
            finalAuthors = authors
    elif "., " in authors:
        search_terms = ["., and ", "., & ", ". and ", ". & "]
        andInAuthors = getIndexOfSubstring(authors, search_terms)[2]
        authors = authors.replace("., ", "#., ")
        if andInAuthors != "":
            authors = authors.replace(andInAuthors, "#., ")
        authors = authors.split("., ")
        authors = [name.replace("#",".") for name in authors]
        authors = [name.replace("..",".") for name in authors]
        for author in authors[:-1]:
            buffer = author.split(", ")
            finalAuthors = finalAuthors + buffer[1] + " " + buffer[0] + " and "
        buffer = authors[-1].split(", ")
        finalAuthors = finalAuthors + buffer[1] + " " +  buffer[0]
    elif ", " in authors:
        search_terms = [", and ", ", & ", " and ", " & "]
        andInAuthors = getIndexOfSubstring(authors, search_terms)[2]
        if andInAuthors != "":  
            authors = authors.replace(andInAuthors, ", ")
        authors = authors.split(", ")
        for i in range(0, len(authors) - 3, 2):
            finalAuthors = finalAuthors + authors[i+1] + " " + authors[i] + " and "
        finalAuthors = finalAuthors + authors[len(authors) - 1] + " " + authors[len(authors) - 2]
    return custom_strip(finalAuthors)

def getAuthors(text):
    
    '''
    Schneidet die Autoren aus dem original Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Autoren und die Autoren
    '''
    
    search_terms = [" and ", ", and ", " & ", ", & ", "., & ", "., and ", ". and ", ". & "]
    
    onlyPunctuation = False
    onlyAnd = False
    authorsDetected = False
    setChainStart = True
    startIndexAuthors = -1
    endIndexAuthors = -1
    chainStartIndex = -1
    changedText = ""
    
    df_PER = getPersonTags(text)
    index_df_PER_List = df_PER.index.values.tolist()
    
    if not df_PER.empty and df_PER["start"].iloc[0] == 0:
        for index in index_df_PER_List:
            if index < len(index_df_PER_List) - 1:
                textBetweenNames = text[df_PER["end"].iloc[index]:df_PER["start"].iloc[index + 1]]
            else:
                textBetweenNames = text[df_PER["end"].iloc[index]:]
            onlyPunctuation = isSpeceficPunctuation(textBetweenNames, ["&"])
            firstStartIndex, firstEndIndex, andTyp = getIndexOfSubstring(textBetweenNames, search_terms)
            onlyAnd = textBetweenNames == andTyp
            if setChainStart: 
                chainStartIndex = df_PER["start"].iloc[index]
                setChainStart = False
            if not onlyPunctuation and not onlyAnd:
                setChainStart = True
                startIndexAuthors = chainStartIndex
                endIndexAuthors = df_PER["end"].iloc[index]
                break
        if startIndexAuthors > -1:
            changedText, author = replaceSubstring(startIndexAuthors, endIndexAuthors, text, ".")
            author = processNames(author)
            return changedText, author
    return text, ""

def getEditors(text):
    
    '''
    Schneidet die Editoren aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Editoren und die ausgeschnittenen Editoren
    '''
    
    search_terms = [" and ", ", and ", " & ", ", & ", "., & ", "., and ", ". and ", ". & "]
    editorRegEx = "\s*(\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors)(\))?\s*"
    onlyPunctuation = False
    onlyAnd = False
    setChainStart = True
    isEditor = False
    startIndexEditors = -1
    endIndexEditors = -1
    chainStartIndex = -1
    startIndexEditorMarker = -1
    endIndexEditorMarker = -1
    
    df_PER = getPersonTags(text)
    index_df_PER_List = df_PER.index.values.tolist()
    if not df_PER.empty:
        for index in index_df_PER_List:
            if index < len(index_df_PER_List) - 1:
                textBetweenNames = text[df_PER["end"].iloc[index]:df_PER["start"].iloc[index + 1]]
            else:
                textBetweenNames = text[df_PER["end"].iloc[index]:]
            onlyPunctuation = isSpeceficPunctuation(textBetweenNames, ["&"])
            firstStartIndex, firstEndIndex, andTyp = getIndexOfSubstring(textBetweenNames, search_terms)
            onlyAnd = textBetweenNames == andTyp
            #if true, that a new chain of Authors begins. An Author Chain is for example "Name1, Name2 and Name3"
            if setChainStart: 
                chainStartIndex = df_PER["start"].iloc[index]
                setChainStart = False
            #If the following if-STatement is true, than the chain has reached an end
            if not onlyPunctuation and not onlyAnd:
                setChainStart = True
                #editors can be the first Part of an literature reference
                textFromStartUntilFirstName = text[0:df_PER["start"].iloc[0]]
                isEditor, startIndexEditorMarker, endIndexEditorMarker = is_Editor(editorRegEx, textFromStartUntilFirstName, 0, False)
                if startIndexEditorMarker == -1:
                    isEditor, startIndexEditorMarker, endIndexEditorMarker = is_Editor(editorRegEx, textBetweenNames, df_PER["end"].iloc[index])                         
                if isEditor:
                    startIndexEditors = chainStartIndex
                    endIndexEditors = df_PER["end"].iloc[index]
                    break
    if startIndexEditors > -1:
        changedText, editor = replaceSubstring(startIndexEditors, endIndexEditors, text, ".")
        editor = processNames(editor)
        if startIndexEditorMarker > -1:
            if startIndexEditorMarker > startIndexEditors:
                startIndexEditorMarker = startIndexEditorMarker - (len(text) - len(changedText))
                endIndexEditorMarker = endIndexEditorMarker - (len(text) - len(changedText))
            changedText, buffer = replaceSubstring(startIndexEditorMarker, endIndexEditorMarker, changedText, ".")
        startIndexIn = 0 
        if startIndexEditorMarker < startIndexEditors:
            startIndexEditors = startIndexEditors - (endIndexEditorMarker - startIndexEditorMarker)
            endIndexEditors = endIndexEditors - (endIndexEditorMarker - startIndexEditorMarker)
        for i in range(startIndexEditors-1, -1, -1):
            if isSpeceficPunctuation(changedText[i], [":", " "]):
                startIndexIn = i + 1
                break
        changedText, replacedEditorMarker = replaceSubstring(startIndexIn, startIndexEditors, changedText, ".")
        return changedText, editor
    return text, ""
    
def getPersonTags(text):
    
    '''
    Gibt die durch Named Entity Recognition erkannten dictionaries vom Typ Person in einem Dataframe zurück
    
    Parameter:
    text: Literaturstring
    
    return: Dataframe der durch Named Entity Recognition erkannten dictionaries vom Typ Person
    '''
    
    ner_tagger = pipeline("ner", aggregation_strategy="simple")
    outputs = ner_tagger(text)
    df_outputs = pd.DataFrame(outputs)
    if not df_outputs.empty: 
        return df_outputs[df_outputs["entity_group"] == "PER"].reset_index(drop=True)
    return pd.DataFrame()

def getORGTag(text, score):
    
    '''
    Gibt die durch Named Entity Recognition erkannten dictionaries vom Typ Organization in einem Dataframe zurück.
    Um in das Dataframe aufgenommen zu werden, muss ein bestimmter Score erreicht sein.
    
    Parameter:
    text: Literaturstring
    score: Schwellenwert zwischen 0 und 1.
    
    return: Dataframe der durch Named Entity Recognition erkannten dictionaries vom Typ Organization
    '''
    
    ner_tagger = pipeline("ner", aggregation_strategy="simple")
    outputs = ner_tagger(text)
    df_outputs = pd.DataFrame(outputs)
    if not df_outputs.empty:
        return df_outputs[(df_outputs["entity_group"] == "ORG") & (df_outputs["score"] >= score)].reset_index(drop=True).tail(1)
    return pd.DataFrame()

def getLOCTag(text):
    
        
    '''
    Gibt die durch Named Entity Recognition erkannten dictionaries vom Typ Location in einem Dataframe zurück
    
    Parameter:
    text: Literaturstring
    
    return: Dataframe der durch Named Entity Recognition erkannten dictionaries vom Typ Location
    '''
    
    ner_tagger = pipeline("ner", aggregation_strategy="simple")
    outputs = ner_tagger(text)
    df_outputs = pd.DataFrame(outputs)
    if not df_outputs.empty:
        return df_outputs[(df_outputs["entity_group"] == "LOC")].reset_index(drop=True)
    return pd.DataFrame()

def getDoi(text):
    
    '''
    Schneidet die DOI aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne DOI und die ausgeschnittene DOI
    '''
    
    doiUrlRegEx1 = "https:\/\/doi\.org(\/[^\s]*)?$"
    doiUrlRegEx2 = "(DOI|doi):\s?(https:\/\/doi\.org)?([^\s]*)+$"
    changedText, doi  = getSubstringByRegEx(text, [doiUrlRegEx1, doiUrlRegEx2])
    httpsDomainRegEx1 = "https:\/\/doi\.org\/"
    httpsDomainRegEx2 = "(DOI|doi):\s?(https:\/\/doi\.org\/)?"
    doi, httpsDomain = getSubstringByRegEx(doi, [httpsDomainRegEx1, httpsDomainRegEx2])
    return changedText, custom_strip(doi)

def getURL(text):
        
    '''
    Schneidet die URL aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne URL und die ausgeschnittene URL
    '''
    
    urlRegEx = "(URL:|url:)?\s*https?://[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?::\d+)?(?:/[^\s]*)?"
    changedText, url = getSubstringByRegEx(text, [urlRegEx])
    urlPrefixRegEx = r"(url:\s*|URL:\s*)"
    url = re.sub(urlPrefixRegEx, '', url).strip()
    return changedText, custom_strip(url)

def getDate(text):
            
    '''
    Schneidet Date aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Date und ausgeschnittene Date
    '''
    
    monthYearRegex = "(January|Jan\.?|February|Feb\.?|March|Mar\.?|April|Apr\.?" \
    "|May|May\.?|June|Jun\.?|July|Jul\.?|August|Aug\.?|September|Sep\.?|Sept\.?|October|" \
    "Oct\.?|November|Nov\.?|December|Dec\.?)\s\d{4}"
    changedText, monthYear  = getSubstringByRegEx(text, [monthYearRegex])
    if monthYear == "":
        yearRegEx1 = "(\.|,)? \(\d{4}\)(\.|,|:)"
        yearRegEx2 = "(\.|,) \d{4}(\.|,|;)"
        changedText, year  = getSubstringByRegEx(text, [yearRegEx1, yearRegEx2])
        return changedText, "", f'{year}'
    monthYear = monthYear.split(' ')
    return changedText, f'{monthYear[0]}', f'{monthYear[1]}'

def getPage(text):
                
    '''
    Schneidet Page aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Page und ausgeschnittene Page
    '''
    
    pageRegEx = "(?:pp\.? )?\d+(-|--|–)\d+"
    changedText, pages = getSubstringByRegEx(text, [pageRegEx])
    if pages != "":
        pages = re.search(r'\d+(-|--|–)\d+', pages).group()
    return changedText, custom_strip(pages)

def getSubstringByRegEx(text, regex = []):
    
    '''
    Prüft für eine Liste von RegEx, ob sie in text vorkommen. Der RegEx, der den Substring mit der grötßen
    Länge ermittelt, kommt zum Zuge. Dieser Substring wird dann aus dem text ausgeschnitten. Ein RegEx wird
    dabei von hinten beginnend in text geprüft und der erste Match zählt.
    
    Parameter:
    text: Literaturstring
    regEx = []: Liste mit RegEx. 
    
    return: Literaturstring ohne Substring und ausgeschnittenen Substring
    '''
    
    startIndex, endIndex, substring = getIndexOfSubstring(text, regex, True)
    changedText, substring = replaceSubstring(startIndex, endIndex, text, "")
    return changedText, custom_strip(substring)

def getVolumeNumber(text):
                    
    '''
    Schneidet Volume und Number aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Volume und Number und ausgeschnittene Volume und Number
    '''
    
    volumeAndNumberRegex = "(\d+\(\d+\)|\d+\.\d+)"
    startIndex, endIndex, substring = getIndexOfSubstring(text, [volumeAndNumberRegex], True)
    if startIndex > -1:
        changedText, volumeNumber = replaceSubstring(startIndex, endIndex, text, "")
        startIndex, endIndex, volume = getIndexOfSubstring(volumeNumber, ["\d+"])
        startIndex, endIndex, number = getIndexOfSubstring(volumeNumber, ["\d+"], True)
        return changedText, volume, number
    else:
        volumeRegEx = "(V|v)ol\. \d+"
        volumeRegEx2 = ", \d+,"
        volumeRegEx3 = ",? \d+(:|\.)"
        number1RegEx = "no\. \d+"
        number2RegEx = "Issue \d+"
        number3RegEx = ", \d+,"
        number4RegEx = "\.\d+"
        startIndex, endIndex, volume = getIndexOfSubstring(text, [volumeRegEx, volumeRegEx2, volumeRegEx3], True)
        changedText, substring = replaceSubstring(startIndex, endIndex, text, "")
        startIndex, endIndex, number = getIndexOfSubstring(changedText, [number1RegEx, number2RegEx, number3RegEx, number4RegEx], True)
        changedText, substring = replaceSubstring(startIndex, endIndex, changedText, "")
        if volume != "":
            volume = re.search(r'\d+', volume).group(0)
        if number != "":
            number = re.search(r'\d+', number).group(0)
        return changedText, volume, number

def getEdition(text):
                        
    '''
    Schneidet Edition aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Edition und ausgeschnittene Edition
    '''
    
    editionRegEx1 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) ed\."
    editionRegEx2 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) edn\."
    changedText, edition = getSubstringByRegEx(text, [editionRegEx1, editionRegEx2])
    if edition != "":
        edition = re.search(r'\d+', edition).group()
    return changedText, custom_strip(edition)

def getAddress(text):
                            
    '''
    Schneidet Address aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Address und ausgeschnittene Address
    '''
    
    df_LOC = getLOCTag(text)
    addressFound = False
    index_df_Loc_List = df_LOC.index.values.tolist()
    textBetweenAddress = ""
    setChainStart = True
    startIndex = 0
    endIndex = 0
    if not df_LOC.empty:
        for index in reversed(index_df_Loc_List):
            if index < len(index_df_Loc_List) and index > 0:
                textBetweenAddress = text[df_LOC["end"].iloc[index-1]:df_LOC["start"].iloc[index]]
            else:
                textBetweenAddress = text[:df_LOC["start"].iloc[index]]
            onlyPunctuation = isSpeceficPunctuation(textBetweenAddress, [])
            if setChainStart: 
                chainEnIndex = df_LOC["end"].iloc[index]
                setChainStart = False
            if not onlyPunctuation:
                startIndex = df_LOC["start"].iloc[index]
                endIndex = chainEnIndex
                break
        address = text[startIndex:endIndex]
        if startIndex > 2 and endIndex < len(text) - 1:
            if isSpeceficPunctuation(text[startIndex - 2]) and isSpeceficPunctuation(text[endIndex + 1]):
                addressFound = True
        else:
            if isSpeceficPunctuation(text[startIndex - 2]):
                addressFound = True
        if addressFound:
            changedText, address = replaceSubstring(startIndex, endIndex, text, "")
            return changedText, custom_strip(address)
    return text, ""  

def getPublisher(text, doi):
                                
    '''
    Schneidet Publisher aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    doi: DOI, um Publisher in externen Datenbanken zu suchen
    
    return: Literaturstring ohne Publisher und ausgeschnittenen Publisher
    '''
    
    publisher = ""
    if doi != "":
        url = f"https://api.crossref.org/works/{doi}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            publisher = data['message'].get('publisher', 'Publisher not found')
    if publisher != "":
        startIndex, endIndex, publisher = getIndexOfSubstring(text, [publisher], True)
        if endIndex < len(text) -1:
            if isSpeceficPunctuation(text[startIndex - 2]) and isSpeceficPunctuation(text[endIndex + 1]):
                changedText, publisher = replaceSubstring(startIndex, endIndex, text, "")
                return changedText, custom_strip(publisher)
        else:
            if isSpeceficPunctuation(text[startIndex - 2]):
                changedText, publisher = replaceSubstring(startIndex, endIndex, text, "")
                return changedText, publisher
        changedText, publisher = replaceSubstring(startIndex, endIndex, text, "")
        if publisher != "":
            return changedText, custom_strip(publisher)
    df_ORG = getORGTag(text, 0.8)
    if not df_ORG.empty:
        startIndex = df_ORG["start"].iloc[0]
        endIndex = df_ORG["end"].iloc[0]
        publisher = text[startIndex:endIndex]
        if endIndex < len(text) -1:
            if isSpeceficPunctuation(text[startIndex - 2]) and isSpeceficPunctuation(text[endIndex + 1]):
                changedText, publisher = replaceSubstring(startIndex, endIndex, text, "")
                return changedText, custom_strip(publisher)
        else:
            if isSpeceficPunctuation(text[startIndex - 2]):
                changedText, publisher = replaceSubstring(startIndex, endIndex, text, "")
                return changedText, custom_strip(publisher)
    return text, ""
    
def getTitel(text):
                                    
    '''
    Schneidet Titel aus dem Literaturstring aus
    
    Parameter:
    text: Literaturstring
    
    return: Literaturstring ohne Titel und ausgeschnittenen Titel
    '''
    
    ignoreCharacters = ["?", "!", "(", ")", "“", "”", "\""]
    text = custom_strip(text, ignoreCharacters)
    ignoreCharacters = ["?", ":", "-", "(", ")", "“", "”", "\""]
    limit = len(text) - 1
    i = 0
    maxIndex = 0
    while i < limit:
        if (i + 2 < limit) and not (text[i] == "," and text[i+1] == " " and not isSpeceficPunctuation(text[i+2])):
            if isSpeceficPunctuation(text[i], ignoreCharacters) and isSpeceficPunctuation(text[i+1], ignoreCharacters):
                text = text[:i] + "." + text[i+2:]
                i = i - 1
                limit = limit - 1
        i = i +1
    
    ignoreCharacters = ["?", "!", "(", ")"]
    if text[0] == "“":
        text = text.rsplit('”', 1)
        return custom_strip(text[0], ignoreCharacters), custom_strip(text[1], ignoreCharacters), ""
    elif text[0] == "\"":
        text = text.rsplit("\"", 1)
        return custom_strip(text[0], ignoreCharacters), custom_strip(text[1], ignoreCharacters), ""
    elif text.count(".") == 1:
        text = text.split(".")
        return custom_strip(text[0]), custom_strip(text[1]), ""
    elif text.count(".") == 2:
        text = text.split(".")
        return custom_strip(text[0]), custom_strip(text[1]), custom_strip(text[2])
    else:
        for index, element in enumerate(text):
            if isSpeceficPunctuation(element, [".", ",", " ", "(", ")", ":"]):
                if maxIndex < index:
                    maxIndex = index
        if maxIndex > 0:
            text = text.split(text[maxIndex])
            return custom_strip(text[0]), custom_strip(text[1]), ""
    if maxIndex == 0 and text.count(",") == 1: 
        text = text.split(",")
        return custom_strip(text[0]), custom_strip(text[1]), ""
    elif maxIndex == 0 and text.count(",") > 1:
        text = text.rsplit(',', 1)
        return custom_strip(text[0]), custom_strip(text[1]), ""
    return custom_strip(text), "", ""
    
    
def getKey(author, year):
                                        
    '''
    Erzeugt einen Key aus dem Nachnamen des ersten Autors und dem Jahr
    
    Parameter:
    author: Autoren
    year: Jahr
    
    return: Erzeugter Schlüssel
    '''
    
    lastNameFirstAuthor = author.split(" and ")[0].strip().split(" ")[-1]
    return f'{lastNameFirstAuthor}_{year}'


def create_bibtex(text):
    address = ""
    author = ""
    booktitle = ""
    chapter = ""
    doi = ""
    edition = ""
    editor = ""
    howpublished = ""
    isbn = ""
    journal = ""
    key = ""
    month = ""
    note = ""
    number = ""
    organization = ""
    pages = ""
    publisher = ""
    school = ""
    series = ""
    title = ""
    url = ""
    volume = ""
    year = ""
    key = ""
    isBook = False
    isProceedings = False
    isInProceedings = False
    isIncollection = False
    isArticle = False
    
    text, author = getAuthors(text)
    text, editor = getEditors(text)
    text, doi = getDoi(text)
    text, url = getURL(text)
    text, month, year = getDate(text)
    text, page = getPage(text)
    text, volume, number = getVolumeNumber(text)
    text, edition = getEdition(text)
    text, address = getAddress(text)
    text, publisher = getPublisher(text, doi)
    school = publisher
    title, booktitle, series = getTitel(text)
    journal = booktitle
    if author != "":
        key = getKey(author, year)
    else:
        key = getKey(editor, year)
    
    bookFields = [author, title, publisher, year, volume, number, \
                  series, address, edition, month, note, key, editor, \
                  howpublished, organization, chapter, pages, isbn, url]
    inproceedingsFields = [author, title, booktitle, year, editor, volume, \
                            number, series, pages, address, month, organization, \
                            publisher, note, key, doi, url]
    proceedingsFields = [title, year, editor, volume, number, series, \
                          address, month, organization, publisher, note, key, doi, url]
    incollectionFields = [author, title, booktitle, publisher, year, editor, \
                           volume, number, series, chapter, pages, address, \
                           edition, month, note, key, doi, url]
    articleFields = [author, title, journal, year, volume, number, \
                      pages, month, note, key, doi, url]
    phdthesisFields = [author, title, publisher, year, address, month, \
                        note, key, doi, url]
    
    bookFieldsString = ["author", "title", "publisher", "year", "volume", "number", \
                  "series", "address", "edition", "month", "note", "key", "editor", \
                  "howpublished", "organization", "chapter", "pages", "isbn", "url"]
    inproceedingsFieldsString  = ["author", "title", "booktitle", "year", "editor", "volume", \
                            "number", "series", "pages", "address", "month", "organization", \
                            "publisher", "note", "key", "doi", "url"]
    proceedingsFieldsString  = ["title", "year", "editor", "volume", "number", "series", \
                          "address", "month", "organization", "publisher", "note", "key", "doi", "url"]
    incollectionFieldsString  = ["author", "title", "booktitle", "publisher", "year", "editor", \
                           "volume", "number", "series", "chapter", "pages", "address", \
                           "edition", "month", "note", "key", "doi", "url"]
    articleFieldsString  = ["author", "title", "journal", "year", "volume", "number", \
                      "pages", "month", "note", "key", "doi", "url"]
    phdthesisFieldsString  = ["author", "title", "school", "year", "address", "month", \
                        "note", "key", "doi", "url"]

    model = "LaLaf93/LiteratureTyp_recognizer"
    classifier = pipeline("text-classification", model=model)
    literatureType = classifier(title + "." + booktitle)[0]['label']
    
    bibTex = "@"
    if literatureType == "book":
        zippedFieldsValues = zip(bookFieldsString, bookFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"book{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    elif literatureType == "proceedings":
        zippedFieldsValues = zip(proceedingsFieldsString, proceedingsFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"proceedings{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    elif literatureType == "inproceedings":
        zippedFieldsValues = zip(inproceedingsFieldsString, inproceedingsFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"inproceedings{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    elif literatureType == "incollection":
        zippedFieldsValues = zip(incollectionFieldsString, incollectionFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"incollection{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    elif literatureType == "article":
        zippedFieldsValues = zip(articleFieldsString, articleFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"article{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    else:
        zippedFieldsValues = zip(phdthesisFieldsString, phdthesisFields)
        zippedList = list(zippedFieldsValues)
        bibTex += f"phdthesis{{{key}, \n"
        for field in zippedList:
            bibTex += f'{field[0]}={{{field[1]}}},\n' 
    
    bibTex += '}'

    return bibTex 



In [25]:
#text="Hinton, G., Bengio, Y., & LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). Physical Review. https://doi.org/10.1109/ICNN.2021.9483948"
#text = "M. A. Nielsen and I. L. Chuang, “Quantum Computation and Quantum Information,” in Handbook of Quantum Information Science, vol. 4, C. H. Bennett and D. P. DiVincenzo, Eds. Berlin, Germany: Springer, 2026, pp. 250–300. doi: 10.1007/springerreference-303198."
#text = "Nielsen, M. A.; Chuang, I. L. Quantum Computation and Quantum Information. In Handbook of Quantum Information Science; Bennett, C. H., DiVincenzo, D. P., Eds.; Quantum Science and Technology; Springer: Berlin, Germany, 2026; Vol. 4, pp 250–300. https://doi.org/10.1007/springerreference-303198."

#BUG: startIndexReplace={-1} ist hier bei getYear! Deswegen doppelter String drin
#text = """Alahmed, Y., Abadla, R., Badri, A. A., & Ameen, N. (2023). “How Does ChatGPT Work” Examining Functionality, To The Creative AI CHATGPT on X’s (Twitter) Platform. 2023 Tenth International Conference on Social Networks Analysis, Management and Security (SNAMS), 1–7. https://doi.org/10.1109/SNAMS60348.2023.10375450"""
#text = "David Mertz, Regular Expression Puzzles and AI Coding Assistants: 24 puzzles solved by the author, with and without assistance from Copilot, ChatGPT and more , Manning, 2023."
#text = """Mohammed Baziyad, Ibrahim Kamel, and Tamer Rabie. 2023. On the Linguistic Limitations of ChatGPT: An Experimental Case Study. In 2023 International Symposium on Networks, Computers and Communications (ISNCC), 1–6. DOI:https://doi.org/10.1109/ISNCC58260.2023.10323661"""
#text = """K. M. Caramancion, "Harnessing the Power of ChatGPT to Decimate Mis/Disinformation: Using ChatGPT for Fake News Detection," 2023 IEEE World AI IoT Congress (AIIoT), Seattle, WA, USA, 2023, pp. 0042-0046, doi: 10.1109/AIIoT58121.2023.10174450."""
#text = """Hinton, G., Bengio, Y., & LeCun, Y. (2021). Deep Learning for Artificial Intelligence? In: J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948"""
#text = """A. Einstein, B. Podolsky, and N. Rosen, “Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?,” Physical Review, vol. 47, no. 10, pp. 777–780, May 1935, doi: 10.1103/PhysRev.47.777."""
#text = """Badaro, G., Saeed, M., & Papotti, P. (2023). Transformers for tabular data representation: a survey of models and applications. Transactions of the Association for Computational Linguistics, 11, pp. 227–249. URL: https://aclanthology.org/2023.tacl-1.14, doi:10.1162/tacl_a_00544"""
#text = """Shamane Siriwardhana, Rivindu Weerasekera, Elliott Wen, Tharindu Kaluarachchi, Rajib Rana, and Suranga Nanayakkara. 2023. Improving the domain adaptation of retrieval augmented generation (RAG) models for open domain question answering. Transactions of the Association for Computational Linguistics 11, (2023), 1–17. URL: https://aclanthology.org/2023.tacl-1.1, doi:10.1162/tacl_a_00530"""
#text = """Mansouri Bigvand, A., Bu, T., & Sarkar, A. (2017). Joint prediction of word alignment with alignment types. Transactions of the Association for Computational Linguistics, 5, pp. 501–514. URL: https://aclanthology.org/Q17-1035, doi:10.1162/tacl_a_00076"""
#text = """Mansouri Bigvand, A., Bu, T., & Sarkar, A. 2017. "Joint Prediction of Word Alignment with Alignment Types." Transactions of the Association for Computational Linguistics, 5: 501–514. URL: https://aclanthology.org/Q17-1035, doi:10.1162/tacl_a_00076."""
#text = """Mansouri Bigvand, A., Bu, T., & Sarkar, A. 2017. ‘Joint Prediction of Word Alignment with Alignment Types." Transactions of the Association forComputational Linguistics, 5: 501-514. URL: https://aclanthology.org/Q17-1035, doi:10.1162/tacl_a_00076."""
#text = """Devika K, Hariprasath .s.b, Haripriya B, Vigneshwar E, Premjith B, and Bharathi Raja Chakravarthi. From dataset to detection: a comprehensive approach to combating Malayalam fake news. In Bharathi Raja Chakravarthi, Ruba Priyadharshini, Anand Kumar Madasamy, Sajeetha Thavareesan, Elizabeth Sherly, Rajeswari Nadarajan, and Manikandan Ravikiran, editors, Proceedings of the Fourth Workshop on Speech, Vision, and Language Technologies for Dravidian Languages, pages 16–23, St. Julian's, Malta, March 2024. Association for Computational Linguistics. URL: https://aclanthology.org/2024.dravidianlangtech-1.3."""
#text = """R, Jairam, G, Jyothish, and B, Premjith. "A few-shot multi-accented speech classification for Indian languages using transformers and LLM's fine-tuning approaches." Proceedings of the Fourth Workshop on Speech, Vision, and Language Technologies for Dravidian Languages. Eds. Chakravarthi, Bharathi Raja, Priyadharshini, Ruba, Madasamy, Anand Kumar, Thavareesan, Sajeetha, Sherly, Elizabeth, Nadarajan, Rajeswari, and Ravikiran, Manikandan. St. Julian's, Malta: Association for Computational Linguistics, 2024. 1–9. URL: https://aclanthology.org/2024.dravidianlangtech-1.1"""
#text = """Rozovskaya, Alla, Roth, Dan, and Sammons, Mark. "Adapting to learner errors with minimal supervision." Computational Linguistics 43.4 (2017): 723–760. https://aclanthology.org/J17-4002, doi:10.1162/COLI_a_00299"""
#text = """Eds. Alonso, Jose M., and Catala, Alejandro. Proceedings of the 1st Workshop on Interactive Natural Language Technology for Explainable Artificial Intelligence (NL4XAI 2019) (Vol. 1, Issue 5, pp. 100–120). 2019. URL: https://aclanthology.org/W19-8400"""
#text = """Hawking, S., Greene, B., Trump, M., & Soy, S. (2025). Advanced Concepts in Theoretical Physics (3rd ed., Vol. 1). Cambridge University Press. https://doi.org/10.1017/example.book.2025"""
#text = """LeCun, Yann, et al. “Gradient-Based Learning Applied to Document Recognition.” Proceedings of the IEEE, vol. 86, no. 11, IEEE, 1998, pp. 2278–324, doi:10.1109/5.726791."""
#text = """Yann LeCun, Léon Bottou, Yoshua Bengio, and Patrick Haffner. 1998. Gradient-based learning applied to document recognition. In Proceedings of the IEEE, IEEE, New York, USA, 2278–2324. DOI:https://doi.org/10.1109/5.726791"""
#text = """Stuart Russell and Peter Norvig. 2016. Artificial Intelligence: A Modern Approach (3rd ed.). Pearson, Upper Saddle River, NJ, USA."""
text = """Neele Falk, Sara Papi, and Mike Zhang. 2024. The Impact of Integration Step on Integrated Gradients. Proceedings of the 18th Conference of the European Chapter of the Association for Computational Linguistics: Student Research Workshop. Association for Computational Linguistics, St. Julian's, Malta, (March 2024). URL: https://aclanthology.org/2024.eacl-srw.0"""
print(create_bibtex(text))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No model was supplied, defaulted

@inproceedings{Falk_2024, 
author={Neele Falk and Sara Papi and Mike Zhang},
title={The Impact of Integration Step on Integrated Gradients},
booktitle={Proceedings of the 18th Conference of the European Chapter of the Association for Computational Linguistics: Student Research Workshop},
year={2024},
editor={},
volume={2024},
number={},
series={},
pages={},
address={St. Julian's, Malta},
month={March},
organization={},
publisher={Association for Computational Linguistics},
note={},
key={Falk_2024},
doi={},
url={https://aclanthology.org/2024.eacl-srw.0},
}


Idee: Nun zunächst Jahr, Volume, Seiten, Edition und URL/DOI extrahieren. Den Rest (also Titel, Publisher, Series), dann nochmal den SpacyParer drüber laufen lassen, weil 

In [19]:
#Test, um Datum Zuverlässig erkannt wird

import spacy
from spacy import displacy



text = "Hinton, G., Bengio, Y., & LeCun, Y. (2021). Deep Learning for Artificial Intelligence, In Proceedings of the IEEE International Conference on Neural Networks, Physical Review."
text = """"What Drives IT Students Toward ChatGPT? Analyzing the Factors Influencing Students' Intention to Use ChatGPT for Educational Purposes," 2024 21st International Multi-Conference on Systems, Signals & Devices (SSD)"""
text = """K. M. Caramancion, "Harnessing the Power of ChatGPT to Decimate Mis/Disinformation: Using ChatGPT for Fake News Detection," 2023 IEEE World AI IoT Congress (AIIoT), Seattle, WA, USA, 2023, pp. 0042-0046, doi: 10.1109/AIIoT58121.2023.10174450."""
text = """. "Harnessing the Power of ChatGPT to Decimate Mis/Disinformation: Using ChatGPT for Fake News Detection," 2023 IEEE World AI IoT Congress (AIIoT), Seattle, WA, USA,  """
text = """Harnessing the Power of ChatGPT to Decimate Mis/Disinformation: Using ChatGPT for Fake News Detection, 2023 IEEE World AI IoT Congress (AIIoT)"""
#text = "dsfsdf London"
print(len(text))
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
# Visualisierung der Abhängigkeiten
displacy.render(doc, style='dep', jupyter=True)
#for token in doc:
#    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
#for ent in doc.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

for token in doc:
    print(f'Text: {token.text}, POS: {token.pos_}, Head: {token.head.text}, Dep: {token.dep_}')

ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)
print(df_outputs)
pos_pipeline = pipeline("token-classification", model="dbmdz/bert-large-cased-finetuned-conll03-english")
results = pos_pipeline(text)
for result in results:
    print(result)
    print("---------------------------------------------------------------------------")
    #print(f"Word: {result['word']}, POS Tag: {result['entity']}")

142


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Text: Harnessing, POS: VERB, Head: Harnessing, Dep: ROOT
Text: the, POS: DET, Head: Power, Dep: det
Text: Power, POS: PROPN, Head: Harnessing, Dep: dobj
Text: of, POS: ADP, Head: Power, Dep: prep
Text: ChatGPT, POS: PROPN, Head: of, Dep: pobj
Text: to, POS: ADP, Head: Harnessing, Dep: prep
Text: Decimate, POS: PROPN, Head: Disinformation, Dep: compound
Text: Mis, POS: PROPN, Head: Disinformation, Dep: nmod
Text: /, POS: SYM, Head: Disinformation, Dep: punct
Text: Disinformation, POS: PROPN, Head: to, Dep: pobj
Text: :, POS: PUNCT, Head: Harnessing, Dep: punct
Text: Using, POS: VERB, Head: Harnessing, Dep: xcomp
Text: ChatGPT, POS: NOUN, Head: Using, Dep: dobj
Text: for, POS: ADP, Head: ChatGPT, Dep: prep
Text: Fake, POS: PROPN, Head: Detection, Dep: compound
Text: News, POS: PROPN, Head: Detection, Dep: compound
Text: Detection, POS: PROPN, Head: for, Dep: pobj
Text: ,, POS: PUNCT, Head: Using, Dep: punct
Text: 2023, POS: NUM, Head: Congress, Dep: nummod
Text: IEEE, POS: ADJ, Head: Con

C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  entity_group     score                   word  start  end
0          ORG  0.879166                   IEEE    108  112
1         MISC  0.753767  World AI IoT Congress    113  134
2          ORG  0.621831                     AI    136  138
3         MISC  0.473681                   ##Io    138  140


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-ORG', 'score': 0.87916553, 'index': 36, 'word': 'IEEE', 'start': 108, 'end': 112}
---------------------------------------------------------------------------
{'entity': 'I-MISC', 'score': 0.9137714, 'index': 37, 'word': 'World', 'start': 113, 'end': 118}
---------------------------------------------------------------------------
{'entity': 'I-MISC', 'score': 0.52765363, 'index': 38, 'word': 'AI', 'start': 119, 'end': 121}
---------------------------------------------------------------------------
{'entity': 'I-MISC', 'score': 0.7182073, 'index': 39, 'word': 'I', 'start': 122, 'end': 123}
---------------------------------------------------------------------------
{'entity': 'I-MISC', 'score': 0.6421149, 'index': 40, 'word': '##o', 'start': 123, 'end': 124}
---------------------------------------------------------------------------
{'entity': 'I-MISC', 'score': 0.91405857, 'index': 41, 'word': '##T', 'start': 124, 'end': 125}
------------------------------------------------

In [14]:
from scholarly import scholarly

# Suche nach einem Autor
search_query = scholarly.search_author('Fernández, R.')
author = next(search_query)

# Laden Sie das komplette Autorenprofil
author = scholarly.fill(author)

# Zeigen Sie einige Informationen über den Autor an
print(f"Name: {author['name']}")
print(f"Affiliation: {author['affiliation']}")

# Zeigen Sie die ersten fünf Veröffentlichungen an
for publication in author['publications'][:5]:
    pub = scholarly.fill(publication)
    print(f"\nTitle: {pub['bib']['title']}")
    print(f"Authors: {pub['bib']['author']}")
    print(f"Year: {pub['bib']['pub_year']}")

Name: T.R. Fernandez Perez Tomei
Affiliation: Researcher IV, Universidade Estadual Paulista

Title: Observation of a new boson at a mass of 125 GeV with the CMS experiment at the LHC
Authors: Serguei Chatrchyan and Vardan Khachatryan and Albert M Sirunyan and Armen Tumasyan and Wolfgang Adam and Ernest Aguilo and Thomas Bergauer and Marko Dragicevic and Janos Erö and Christian Fabjan and M Friedl and Rudolf Frühwirth and VM Ghete and J Hammer and M Hoch and N Hörmann and J Hrubec and M Jeitler and W Kiesenhofer and V Knünz and M Krammer and I Krätschmer and D Liko and W Majerotto and I Mikulec and M Pernicka and B Rahbaran and C Rohringer and H Rohringer and R Schöfbeck and J Strauss and F Szoncsó and A Taurok and W Waltenberger and G Walzel and E Widl and C-E Wulz and I Emeliantchik and V Makarenko and N Shumeiko and A Solin and R Stefanovitch and J Suarez Gonzalez and A Fedorov and M Korzhik and O Missevitch and R Zuyeuski and M Bansal and S Bansal and W Beaumont and Tom Cornelis and


Title: Evidence for the 125 GeV Higgs boson decaying to a pair of τ leptons
Authors: Serguei Chatrchyan and Vardan Khachatryan and Albert M Sirunyan and Armen Tumasyan and Wolfgang Adam and Thomas Bergauer and Marko Dragicevic and Janos Erö and Christian Fabjan and Markus Friedl and Rudolf Fruehwirth and Vasile Mihai Ghete and Christian Hartl and Natascha Hörmann and Josef Hrubec and Manfred Jeitler and Wolfgang Kiesenhofer and Valentin Knünz and Manfred Krammer and Ilse Krätschmer and Dietrich Liko and Ivan Mikulec and Dinyar Rabady and Babak Rahbaran and Herbert Rohringer and Robert Schöfbeck and Josef Strauss and Anton Taurok and Wolfgang Treberer-Treberspurg and Wolfgang Waltenberger and C-E Wulz and N Shumeiko and S Alderweireldt and M Bansal and S Bansal and T Cornelis and EA De Wolf and X Janssen and A Knutsson and S Luyckx and S Ochesanu and B Roland and R Rougny and H Van Haevermaet and P Van Mechelen and A Van Spilbeeck and F Blekman and S Blyweert and J D’Hondt and N Heracl

In [1]:
"Hallo"[0:5]

'Hallo'